In [1]:
import numpy as np
import torch
from omegaconf import OmegaConf, DictConfig
from scipy.spatial.distance import cosine

from transformers import BertTokenizer

from models import compose_model
from text_processing import *
from utils import *

In [2]:
emotions = "admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral".split(',')
emotions_map = {i: em for i, em in enumerate(emotions)}

In [3]:
config = """
model:
  recipe:
    # 'BERT' or 'FastText'
    word_embedding: 'BERT'
    multi_label: True
    use_context: False
    # 'sep' or 'cls-concat' or 'emo-concat'
    context_type: 'cls-concat'
    freeze_emotion_embedding: False

  classes_num: 28
  lstm:
    hidden_size: 128
    num_layers: 1
    bidirectional: True
    # 'sum' or 'last-sum' or 'last-concat'
    output_assemble_type: 'sum'
    dropout: 0.0
  classifier:
    hidden_sizes: [ ]
    dropout_p: 0.25

  emotion_embedding_size: 128
  emotion_dropout_p: 0.25

bert:
  checkpoint_path: '/home/asapozhnikov/projects/emotions/checkpoints/uncased_L-12_H-768_A-12/'
  embedding_size: 768
  text_preprocessing: True
  finetune: False
"""

hparams = OmegaConf.create(config)

In [4]:
model = compose_model(hparams.model.recipe, hparams)

Some weights of the model checkpoint at /home/asapozhnikov/projects/emotions/checkpoints/uncased_L-12_H-768_A-12/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
checkpoint_path = '/home/asapozhnikov/projects/emotions/go_emotions_bert_base.ckpt'
model = load_checkpoint(checkpoint_path, model, None)[0]

Loading checkpoint '/home/asapozhnikov/projects/emotions/outputs/2022-06-13/22-05-18_sum_weights_oversampled/checkpoints/sum-weighted__iter_19991__f1_macro_0.46.ckpt'
Loaded checkpoint '/home/asapozhnikov/projects/emotions/outputs/2022-06-13/22-05-18_sum_weights_oversampled/checkpoints/sum-weighted__iter_19991__f1_macro_0.46.ckpt' without BERT from iteration 19991


In [6]:
bert_tokenizer = BertTokenizer.from_pretrained(hparams.bert.checkpoint_path)

In [7]:
def get_embedding_and_classification(model, texts):
    p = TextProcessingPipeline.get_standard_pipeline()
    texts = [p(text) for text in texts]
    tokens  = bert_tokenizer(texts, return_tensors="pt", padding=True)
    lengths = torch.sum(tokens.attention_mask, dim=1)
    emotions, embeddings = model.inferense((tokens, lengths))
    emotions = torch.sigmoid(emotions).round()
    emotions_mapped = []
    for sample_i in range(emotions.shape[0]):
        cur_emotions = []
        for i in range(emotions.shape[1]):
            if emotions[sample_i][i]:
                cur_emotions.append(emotions_map[i])
        emotions_mapped.append(cur_emotions)
    return emotions_mapped, embeddings.detach().numpy()

In [9]:
emotions, embeddings = get_embedding_and_classification(model, [
    "i love you",
    "i hate you",
    "sooooo cuuute doggie <3"
])
emotions

[['love'], ['anger'], ['admiration']]

# Emotion Embedding similarity

In [12]:
embeddings_love = embeddings[0] / np.linalg.norm(embeddings[0])
embeddings_angr = embeddings[1] / np.linalg.norm(embeddings[1])
embeddings_admi = embeddings[2] / np.linalg.norm(embeddings[2])

In [14]:
1 - cosine(embeddings_love, embeddings_angr)

0.30952560901641846

In [15]:
1 - cosine(embeddings_love, embeddings_admi)

0.7504923343658447

In [16]:
1 - cosine(embeddings_angr, embeddings_admi)

-0.02869943529367447

# Emotion Embedding math operations

In [11]:
emotions, embeddings = get_embedding_and_classification(model,
[
    "I dream to be an astronaut! I'm so happy that I entered the NASA internship",
    "I WANT PIZZA",
    "That service is awfull. I would reply",
])
emotions

[['desire', 'joy'], ['desire'], ['disgust']]

In [15]:
emb = embeddings[0] - embeddings[1]

emotions = model.predict_by_emotion_embedding(torch.tensor([emb]))[0]
emotions = torch.sigmoid(emotions).round()
emotions_mapped = []
for sample_i in range(emotions.shape[0]):
    cur_emotions = []
    for i in range(emotions.shape[1]):
        if emotions[sample_i][i]:
            cur_emotions.append(emotions_map[i])
    emotions_mapped.append(cur_emotions)

print(f"['desire', 'joy'] - ['desire'] = {emotions_mapped[0]}")

['desire', 'joy'] - ['desire'] = ['joy']
